In [1]:
import pandas as pd

lab = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\D1_BD_PADRIS_regicor\AP_Laboratoris.csv", sep = "|")

C:\Users\mhuertas\AppData\Local\Temp\ipykernel_21148\1498111611.py:3: DtypeWarning: Columns (3,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  lab = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\D1_BD_PADRIS_regicor\AP_Laboratoris.csv", sep = "|")


In [2]:
from collections import defaultdict
from utils.functions import preprocessing, add_commons, processing

In [3]:
lab

,codi_p,Any_prova,Data_prova,peticio_id,lab_prova_c,lab_prova,lab_resultat,unitat_mesura,ref_min,ref_max
0,10254,2010,17/08/2010,81705546520,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
1,7314,2011,01/08/2011,80100100846,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
2,8871,2008,17/03/2008,31774972,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
3,3241,2008,13/05/2008,51354073,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
4,11405,2010,20/04/2010,42005701677,H11740,BASÒFILS;C.NOM.,0,K/mcL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7825016,9255,2016,01/09/2016,90105546769,H11840,BASÒFILS;FR.NOM. (leucòcits),",5",%,0,2
7825017,10726,2016,04/11/2016,110400100272,H11840,BASÒFILS;FR.NOM. (leucòcits),",5",%,0,2
7825018,7691,2017,13/12/2017,121305613907,H11840,BASÒFILS;FR.NOM. (leucòcits),",5",%,0,2
7825019,11051,2017,04/10/2017,100405547798,H11840,BASÒFILS;FR.NOM. (leucòcits),",5",%,0,2


In [4]:
preprocessing_df = preprocessing(lab)

u:\Estudis\B52_CORDELIA\Analisis\PADRIS\01_Cleaning\source\lab\utils\functions.py:47: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  .assign(Data_prova=pd.to_datetime(chunk['Data_prova'], errors='coerce').dt.strftime('%d/%m/%Y'))


In [5]:
preprocessing_df

,codi_p,Any_prova,Data_prova,peticio_id,lab_prova_c,lab_prova,lab_resultat,unitat_mesura,ref_min,ref_max
0,10254,2010,17/08/2010,81705546520,H11740,BASÒFILS;C.NOM.,0,10^3/µl,NaN,NaN
1,7314,2011,01/08/2011,80100100846,H11740,BASÒFILS;C.NOM.,0,10^3/µl,NaN,NaN
2,8871,2008,17/03/2008,31774972,H11740,BASÒFILS;C.NOM.,0,10^3/µl,NaN,NaN
3,3241,2008,13/05/2008,51354073,H11740,BASÒFILS;C.NOM.,0,10^3/µl,NaN,NaN
4,11405,2010,20/04/2010,42005701677,H11740,BASÒFILS;C.NOM.,0,10^3/µl,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7825016,9255,2016,01/09/2016,90105546769,H11840,BASÒFILS;FR.NOM. (leucòcits),0.5,%,0.0,2.0
7825017,10726,2016,04/11/2016,110400100272,H11840,BASÒFILS;FR.NOM. (leucòcits),0.5,%,0.0,2.0
7825018,7691,2017,13/12/2017,121305613907,H11840,BASÒFILS;FR.NOM. (leucòcits),0.5,%,0.0,2.0
7825019,11051,2017,04/10/2017,100405547798,H11840,BASÒFILS;FR.NOM. (leucòcits),0.5,%,0.0,2.0


In [14]:
label_counts = defaultdict(lambda: defaultdict(int))
unit_counts = defaultdict(lambda: defaultdict(int))

# Update both label_counts and unit_counts in one loop
for prova, group in preprocessing_df.groupby('lab_prova_c'):
    label_counts[prova].update(group['lab_prova'].value_counts().to_dict())
    unit_counts[prova].update(group['unitat_mesura'].value_counts().to_dict())

In [15]:
def add_commons(chunk, label_counts, unit_counts):
    """ Updates label and unit counts and adds columns for the most common label and unit """

    # Group by 'lab_prova_c' to get value counts for 'lab_prova'
    label_counts_per_group = chunk.groupby('lab_prova_c')['lab_prova'].value_counts()

    # Update label_counts dictionary with the aggregated counts
    label_counts_per_group.groupby(level=0).apply(lambda x: label_counts[x.name].update(x.to_dict()))

    # Get the most common 'lab_prova' for each 'lab_prova_c'
    most_common_labels = label_counts_per_group.groupby(level=0).idxmax().apply(lambda x: x[1])

    # If unit_counts is required, process similarly to label_counts
    if unit_counts is not None:
        unit_counts_per_group = chunk.groupby('lab_prova_c')['unitat_mesura'].value_counts()
        unit_counts_per_group.groupby(level=0).apply(lambda x: unit_counts[x.name].update(x.to_dict()))
        most_common_units = unit_counts_per_group.groupby(level=0).idxmax().apply(lambda x: x[1])
        chunk['common_unit'] = chunk['lab_prova_c'].map(most_common_units)
    else:
        most_common_units = None

    # Map the most common labels back to the 'lab_prova' column
    chunk['lab_prova'] = chunk['lab_prova_c'].map(most_common_labels)
    

    return chunk


In [16]:
add_commons_df =  add_commons(preprocessing_df, label_counts, unit_counts)

In [20]:
add_commons_df

,codi_p,Any_prova,Data_prova,peticio_id,lab_prova_c,lab_prova,lab_resultat,unitat_mesura,ref_min,ref_max,common_unit
0,10254,2010,17/08/2010,81705546520,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,10^3/µl,NaN,NaN,10^3/µl
1,7314,2011,01/08/2011,80100100846,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,10^3/µl,NaN,NaN,10^3/µl
2,8871,2008,17/03/2008,31774972,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,10^3/µl,NaN,NaN,10^3/µl
3,3241,2008,13/05/2008,51354073,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,10^3/µl,NaN,NaN,10^3/µl
4,11405,2010,20/04/2010,42005701677,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,10^3/µl,NaN,NaN,10^3/µl
...,...,...,...,...,...,...,...,...,...,...,...
7825016,9255,2016,01/09/2016,90105546769,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,0.5,%,0.0,2.0,%
7825017,10726,2016,04/11/2016,110400100272,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,0.5,%,0.0,2.0,%
7825018,7691,2017,13/12/2017,121305613907,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,0.5,%,0.0,2.0,%
7825019,11051,2017,04/10/2017,100405547798,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,0.5,%,0.0,2.0,%


In [21]:
import pandas as pd
import os
import re
import numpy as np


def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

def generate_summary(df):
    """Generate summary statistics table"""
    # Convert lab_resultat to numeric
    df['lab_resultat'] = pd.to_numeric(df['lab_resultat'], errors='coerce')

    # Drop rows with NaN in lab_resultat
    df = df.dropna(subset=['lab_resultat'])

    # Group by lab_prova, lab_prova_c, and unitat_mesura
    grouped = df.groupby(['lab_prova', 'lab_prova_c', 'unitat_mesura'])

    # Calculate statistics for each group
    summary_df = grouped.agg(
        count=('lab_resultat', 'size'),
        mean=('lab_resultat', 'mean'),
        std=('lab_resultat', 'std'),
        median=('lab_resultat', 'median'),
        q1=('lab_resultat', q1),
        q3=('lab_resultat', q3),
        max=('lab_resultat', 'max'),
        min=('lab_resultat', 'min')
    ).reset_index()

    # Round numerical columns to 2 decimal places
    summary_df = summary_df.round({
        'mean': 2,
        'std': 2,
        'median': 2,
        'q1': 2,
        'q3': 2,
        'min': 2,
        'max': 2
    })

    # Order the dataframe
    summary_df = summary_df.sort_values(by="count", ascending=False)

    return summary_df

In [22]:
summary = generate_summary(add_commons_df)

TypeError: _UFuncNoLoopError.__init__() missing 1 required positional argument: 'dtypes'

In [12]:
process_df

,codi_p,Any_prova,Data_prova,peticio_id,lab_prova_c,lab_prova,lab_resultat,unitat_mesura,ref_min,ref_max,common_label,common_units,transformed_value
38088357,27186.0,2017,02/10/2017,100205461703,Q32685,GLUCOSA; C.SUBST. (sèrum),85.0,mg/dl,70.00,105.00,GLUCOSA (SÈRUM); C.SUBST.,mg/dl,85.0
30376755,11509.0,2014,26/09/2014,92605703877,Q30385,FOSFATASA ALCALINA; C.CAT. (sèrum),70.0,ui/l,40.00,129.00,FOSFATASA ALCALINA (SÈRUM); C.CAT.,ui/l,70.0
38131915,21249.0,2022,23/04/2022,64167510746,Q32685,GLUCOSA-SÈRUM,169.0,mg/dl,82.00,82.00,GLUCOSA (SÈRUM); C.SUBST.,mg/dl,169.0
16064314,55595.0,2016,11/02/2016,2016021190204900,H10940,NEUTRÒFILS;C.NOM. (leucòcits),4.0,10^9/l,1.50,6.50,NEUTRÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,10^3/µl,4.0
24920749,67307.0,2020,19/05/2020,200519680022189,H10872,LEUCÒCITS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,3.5,10^3/µl,4.50,11.00,LEUCÒCITS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,10^3/µl,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17934774,24972.0,2023,07/12/2023,231207408259586,Q71375,"PH, IÓ HIDRÒGEN (S.VENOSA). «PART DE CONJUNT»",7.37,NaN,7.32,7.42,"PH, IÓ HIDRÒGEN (S.VENOSA). «PART DE CONJUNT»",mmhg,7.37
23897593,60558.0,2017,18/07/2017,170718651001410,H10272,ERITRÒCITS;C.NOM. (sang),4.55,10^12/l,4.00,5.40,ERITRÒCITS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,10^3/µl,4.55
37144456,54235.0,2008,05/11/2008,317045,Q13285,COLESTEROL; C.SUBST.,199.0,mg/dl,100.00,240.00,COLESTEROL; C.SUBST. (sèrum),mg/dl,199.0
10236180,57360.0,2020,28/05/2020,202005285843649,Q30385,FOSFATASA ALCALINA (SÈRUM); C.CAT.,109.0,ui/l,40.00,129.00,FOSFATASA ALCALINA (SÈRUM); C.CAT.,ui/l,109.0


In [12]:
import pandas as pd
file_path = r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\01_BD_PADRIS_clean\AP_Laboratoris.csv"
lab = pd.read_csv(file_path, sep="|", nrows = 48214634)

C:\Users\mhuertas\AppData\Local\Temp\ipykernel_2680\1151461955.py:3: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  lab = pd.read_csv(file_path, sep="|", nrows = 48214634)


In [13]:
lab

,codi_p,Any_prova,Data_prova,peticio_id,lab_prova_c,lab_prova,lab_resultat,unitat_mesura,ref_min,ref_max
0,25953,2009,24/02/2009,22456301,H11740,BASÒFILS;C.NOM.,0,k/mcl,NaN,NaN
1,47733,2009,12/05/2009,51200055761,H11740,BASÒFILS;C.NOM.,0,k/mcl,NaN,NaN
2,10254,2010,17/08/2010,81705546520,H11740,BASÒFILS;C.NOM.,0,k/mcl,NaN,NaN
3,37448,2009,14/10/2009,101400054993,H11740,BASÒFILS;C.NOM.,0,k/mcl,NaN,NaN
4,30960,2009,03/04/2009,40353174,H11740,BASÒFILS;C.NOM.,0,k/mcl,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
48214629,14886,2014,05/08/2014,80505546156,Q15185,CREATININI; C.SUBST. (sèrum),0.63,mg/dl,0.5,0.9
48214630,17243,2017,09/10/2017,100905625721,H11840,BASÒFILS;FR.NOM. (leucòcits),0.5,%,0.0,2.0
48214631,1608,2016,19/01/2016,11905492936,H11840,BASÒFILS;FR.NOM. (leucòcits),0.5,%,0.0,2.0
48214632,20552,2016,05/10/2016,100500109080,H11840,BASÒFILS;FR.NOM. (leucòcits),0.6,%,0.0,2.0
